In [4]:
import numpy as np

In [10]:
import krotov

In [15]:
import qutip

In [11]:
from src.ham import ryd2_hamiltonian, ket

In [6]:
GHz = 2 * np.pi
MHz = 1e-3 * GHz
ns = 1

In [8]:
H = ryd2_hamiltonian(E1=(9.1 * GHz), Δ1=(1.273 * GHz), Δ2=0, u=(57.26 * MHz), Ω_B_left=Ω_B_left, Ω_B_right=Ω_B_right, Ω_R_left=Ω_R_left, Ω_R_right=Ω_R_right)

In [12]:
basis_states = [ket(label) for label in ('00', '01', '10', '11')]

In [22]:
cphase = qutip.qip.operations.cphase(np.pi)

In [23]:
cphase

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0. -1.]]

In [24]:
objectives = krotov.gate_objectives(basis_states, cphase, H)

In [25]:
objectives

[Objective[|Ψ₀(4⊗4)⟩ to |Ψ₀(4⊗4)⟩ via [H₀[4⊗4,4⊗4], [H₁[4⊗4,4⊗4], u₁(t)], [H₂[4⊗4,4⊗4], u₂(t)], [H₃[4⊗4,4⊗4], u₁(t)], [H₄[4⊗4,4⊗4], u₁(t)]]],
 Objective[|Ψ₁(4⊗4)⟩ to |Ψ₁(4⊗4)⟩ via [H₀[4⊗4,4⊗4], [H₁[4⊗4,4⊗4], u₁(t)], [H₂[4⊗4,4⊗4], u₂(t)], [H₃[4⊗4,4⊗4], u₁(t)], [H₄[4⊗4,4⊗4], u₁(t)]]],
 Objective[|Ψ₂(4⊗4)⟩ to |Ψ₂(4⊗4)⟩ via [H₀[4⊗4,4⊗4], [H₁[4⊗4,4⊗4], u₁(t)], [H₂[4⊗4,4⊗4], u₂(t)], [H₃[4⊗4,4⊗4], u₁(t)], [H₄[4⊗4,4⊗4], u₁(t)]]],
 Objective[|Ψ₃(4⊗4)⟩ to |Ψ₄(4⊗4)⟩ via [H₀[4⊗4,4⊗4], [H₁[4⊗4,4⊗4], u₁(t)], [H₂[4⊗4,4⊗4], u₂(t)], [H₃[4⊗4,4⊗4], u₁(t)], [H₄[4⊗4,4⊗4], u₁(t)]]]]

In [31]:
T = 800

In [32]:
tlist = np.linspace(0, T, 1000)

In [33]:
objectives[0].mesolve(tlist)

Result object with sesolve data.
--------------------------------
states = True
num_collapse = 0

In [34]:
def update_shape(t):
    return krotov.shapes.flattop(t, 0, T, t_rise=0.1*T)

In [35]:
pulse_options = {
    Ω_B_left: dict(lambda_a=1, update_shape=update_shape),
    Ω_B_right: dict(lambda_a=1, update_shape=update_shape),
}

In [ ]:
res_opt = krotov.optimize_pulses(
    objectives,
    pulse_options,
    tlist,
    propagator=krotov.propagators.expm,
    chi_constructor=krotov.functionals.chis_sm,
    info_hook=krotov.info_hooks.print_table(J_T=krotov.functionals.J_T_sm),
)